# Exp-01: Chunking Strategy Optimization
- **Date**: 2026-02-05
- **Goal**: RFP 문서에 최적화된 Chunk Size 및 Table Strategy 도출
- **Focus**: Context Recall Maximization (Target: 0.85+)

## 변경 이력
- v2: Human-labeled Ground Truth 기반 실제 평가로 전환 (Mock Score 제거)

## 1. 실험 설계 (Experiment Design)
### 1.1 변수 (Variables)
- **Independent (실험군)**:
  - **Chunk Size**: [500, 1000, 2000]
  - **Table Strategy**: [`Text` (단순), `Layout` (구조 보존)]
- **Controlled (통제 변수)**:
  - Retrieval: Hybrid Search (Default)
  - Top-K: 10

### 1.2 가설 (Hypothesis)
- RFP는 표(Table)에 중요한 스펙 정보가 많으므로, `Layout` 전략이 `Text`보다 Recall이 높을 것이다.
- 문맥이 긴 문서 특성상 500자보다는 1000자가 유리하나, 2000자는 검색 노이즈가 발생할 것이다.


## 2. 환경 설정 (Setup)
데이터 로더 및 필요 라이브러리를 임포트합니다.


In [ ]:
import sys
import os
import time
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 루트 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Any

# BidFlow 모듈
from bidflow.ingest.loader import RFPLoader
from bidflow.ingest.storage import DocumentStore, VectorStoreManager
from bidflow.retrieval.hybrid_search import HybridRetriever
from bidflow.eval.ragas_runner import RagasRunner
from bidflow.parsing.pdf_parser import PDFParser

%matplotlib inline
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

print("[System] 모듈 로드 완료")

## 3. Baseline 설정
- **Setting**: Chunk 500 / Text Strategy
- **Target Metric**: Context Recall
- **Budget**: Latency < 1.0s


## 4. 데이터 및 파이프라인 (Data & Pipeline)
- **Dataset**: 실제 RFP 문서 샘플 (`data/raw/files`)
- **Pipeline**: `RFPLoader` -> `Chunking` -> `Ragas Eval` -> `Report`


In [ ]:
# 경로 설정
DATA_DIR = "../data/raw/files"
META_PATH = "../data/raw/data_list.csv"
GOLDEN_TESTSET_PATH = "../data/experiments/golden_testset.csv"
CHROMA_BASE_PATH = "../data/chroma_exp"  # 실험용 별도 DB

# 실험 변수 정의
CHUNK_SIZES = [500, 1000, 2000]
TABLE_STRATEGIES = ["text", "layout"]

# 1. Golden Testset 로드 (Human-labeled)
print("=" * 50)
print("1. Golden Testset 로드")
print("=" * 50)

if os.path.exists(GOLDEN_TESTSET_PATH):
    golden_df = pd.read_csv(GOLDEN_TESTSET_PATH)
    # 템플릿 데이터 필터링 (실제 레이블된 데이터만)
    golden_df = golden_df[~golden_df['ground_truth'].str.contains(r'\[여기에', na=False)]
    
    if len(golden_df) == 0:
        print("[Warning] Golden Testset이 비어있습니다!")
        print(f"   - {GOLDEN_TESTSET_PATH} 파일을 열어 실제 정답을 입력하세요.")
        print("   - question: 평가할 질문")
        print("   - ground_truth: 문서에서 찾을 수 있는 정답")
        TESTSET_READY = False
    else:
        print(f"[OK] {len(golden_df)}개 테스트 케이스 로드됨")
        print(golden_df[['question', 'category']].head())
        TESTSET_READY = True
else:
    print(f"[Error] Golden Testset 파일이 없습니다: {GOLDEN_TESTSET_PATH}")
    TESTSET_READY = False

# 2. 샘플 PDF 파일 찾기
print("\n" + "=" * 50)
print("2. 샘플 PDF 파일 검색")
print("=" * 50)

SAMPLE_FILE = None
if os.path.exists(DATA_DIR):
    pdfs = [f for f in os.listdir(DATA_DIR) if f.lower().endswith('.pdf')]
    if pdfs:
        SAMPLE_FILE = os.path.join(DATA_DIR, pdfs[0])
        print(f"[OK] 샘플 PDF: {pdfs[0]}")
    else:
        print("[Error] PDF 파일을 찾을 수 없습니다.")
else:
    print(f"[Error] 데이터 디렉토리가 없습니다: {DATA_DIR}")

# 결과 저장용
results = []
chunk_stats = []  # 청크 통계

In [ ]:
# [Quick Fix] Reload RagasRunner explicitly to apply simplified transforms
import importlib
import bidflow.eval.ragas_runner
importlib.reload(bidflow.eval.ragas_runner)
from bidflow.eval.ragas_runner import RagasRunner
runner = RagasRunner()

TESTSET_PATH = "../data/experiments/golden_testset.csv"

if os.path.exists(TESTSET_PATH):
    print("Loading existing Golden Testset...")
    testset_df = pd.read_csv(TESTSET_PATH)
else:
    print("Generating Golden Testset (This may take a while)...")
    if SAMPLE_FILE:
        # 1. Baseline Ingestion
        loader.vec_manager.clear() # Clean State
        with open(SAMPLE_FILE, "rb") as f:
             # Default: chunk=1000, table=text (표준)
             doc_hash = loader.process_file(f, os.path.basename(SAMPLE_FILE), chunk_size=1000, table_strategy="text")
        
        # 2. Load Parsed Documents for Ragas
        # Ragas needs LangChain Document list. We can reconstruct it from the loaded RFPDocument
        rfp_doc = loader.doc_store.load_document(doc_hash)
        lc_docs = []
        from langchain_core.documents import Document
        for chunk in rfp_doc.chunks:
            lc_docs.append(Document(page_content=chunk.text, metadata=chunk.metadata))
            
        # 3. Generate
        testset_df = runner.generate_testset(lc_docs, test_size=5) # 5 Questions
        testset_df.to_csv(TESTSET_PATH, index=False)
        print(f"Golden Testset saved to {TESTSET_PATH}")
    else:
        print("No Sample File found! Cannot generate testset.")
        testset_df = pd.DataFrame()

display(testset_df.head(2))

## 5. 평가 지표 (Metrics)
- **Context Recall**: 정답(Ground Truth)이 검색된 청크 내에 존재하는지 여부. (RFP 분석의 핵심)
- **Latency (p95)**: 검색 응답 속도.


## 6. 실험 실행 (Execution)
정의된 변수 조합(Grid Search)에 따라 파이프라인을 실행합니다.


In [ ]:
def run_single_experiment(
    pdf_path: str,
    chunk_size: int,
    table_strategy: str,
    golden_df: pd.DataFrame,
    exp_chroma_path: str
) -> Dict[str, Any]:
    """
    단일 설정에 대해 Chunking -> Indexing -> Retrieval 평가 수행
    """
    # ===== Fix 1: 모든 필요한 임포트 추가 =====
    import numpy as np
    import gc
    import time
    import shutil
    from langchain_openai import OpenAIEmbeddings
    from langchain_chroma import Chroma
    from langchain_core.documents import Document
    
    # ===== Fix 2: config_name 정의 (맨 앞에!) =====
    config_name = f"chunk={chunk_size}_table={table_strategy}"
    
    print(f"\n{'='*60}")
    print(f"[Experiment] {config_name}")
    print(f"{'='*60}")
    
    def robust_rmtree(path):
        """Windows 파일 락 대응 삭제"""
        if not os.path.exists(path): 
            return
        for _ in range(5):
            try:
                shutil.rmtree(path)
                return
            except PermissionError:
                gc.collect()
                time.sleep(1)
            except Exception as e:
                print(f'Warning: rmtree failed: {e}')
                return
        print(f'Failed to delete {path} after retries.')
    
    # ===== Fix 3: 실험 DB 초기화 (robust_rmtree 호출) =====
    robust_rmtree(exp_chroma_path)
    os.makedirs(exp_chroma_path, exist_ok=True)
    
    # 1. PDF 파싱
    start_time = time.perf_counter()
    
    parser = PDFParser()
    chunks = parser.parse(
        pdf_path,
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size * 0.1),
        table_strategy=table_strategy
    )
    
    parse_time = time.perf_counter() - start_time
    print(f"  파싱 완료: {len(chunks)}개 청크 ({parse_time:.2f}s)")
    
    # 청크 통계
    chunk_lengths = [len(c.text) for c in chunks]
    chunk_stat = {
        "config": config_name,
        "chunk_size": chunk_size,
        "table_strategy": table_strategy,
        "num_chunks": len(chunks),
        "avg_length": float(np.mean(chunk_lengths)),
        "std_length": float(np.std(chunk_lengths)),
        "min_length": int(np.min(chunk_lengths)),
        "max_length": int(np.max(chunk_lengths)),
    }
    
    # 2. VectorStore 구축
    start_time = time.perf_counter()
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vector_db = Chroma(
        persist_directory=exp_chroma_path,
        embedding_function=embeddings,
        collection_name="exp_chunking"
    )
    
    lc_docs = [
        Document(
            page_content=c.text,
            metadata={"chunk_id": c.chunk_id, "page": c.page_no}
        )
        for c in chunks
    ]
    vector_db.add_documents(lc_docs)
    
    index_time = time.perf_counter() - start_time
    print(f"  인덱싱 완료 ({index_time:.2f}s)")
    
    # 3. Retrieval 및 Context Recall 평가
    start_time = time.perf_counter()
    
    retriever = vector_db.as_retriever(search_kwargs={"k": 10})
    
    questions = []
    ground_truths = []
    contexts_list = []

    for _, row in golden_df.iterrows():
        question = row['question']
        ground_truth = str(row['ground_truth'])
        
        retrieved_docs = retriever.invoke(question)
        contexts_list.append([doc.page_content for doc in retrieved_docs])
        questions.append(question)
        ground_truths.append(ground_truth)

    # ===== Fix 4: Ragas v0.2+ API 호환 키 사용 =====
    from bidflow.eval.ragas_runner import RagasRunner
    from ragas import evaluate
    from ragas.metrics import ContextRecall
    from datasets import Dataset

    try:
        # Ragas v0.2+ 키 이름: user_input, reference, retrieved_contexts
        data = {
            "user_input": questions,
            "reference": ground_truths,
            "retrieved_contexts": contexts_list
        }
        dataset = Dataset.from_dict(data)
        runner = RagasRunner()
        
        print(f"  Ragas Evaluating {len(dataset)} items...")
        eval_result = evaluate(
            dataset,
            metrics=[ContextRecall(llm=runner.llm)],
            llm=runner.llm,
            embeddings=runner.embeddings,
            raise_exceptions=False
        )
        
        raw_metric = eval_result["context_recall"]
        if isinstance(raw_metric, (list, tuple)):
            avg_recall = float(np.mean(raw_metric))
        elif hasattr(raw_metric, "mean"):
            avg_recall = float(raw_metric.mean())
        else:
            avg_recall = float(raw_metric) if raw_metric is not None else 0.0
            
    except Exception as e:
        print(f"  Ragas Error: {e}, falling back to 0")
        avg_recall = 0.0

    retrieval_time = time.perf_counter() - start_time
    print(f"  검색 테스트 완료: Context Recall = {avg_recall:.4f} ({retrieval_time:.2f}s)")
    
    # 4. 결과 반환
    result = {
        "config": config_name,
        "chunk_size": chunk_size,
        "table_strategy": table_strategy,
        "context_recall": avg_recall,
        "num_chunks": len(chunks),
        "parse_time": float(parse_time),
        "index_time": float(index_time),
        "retrieval_time": float(retrieval_time),
        "latency_total": float(parse_time + index_time + retrieval_time),
    }
    
    return result, chunk_stat


# 실험 실행
if SAMPLE_FILE and TESTSET_READY:
    print("\n" + "=" * 60)
    print("실험 시작: Grid Search")
    print("=" * 60)
    
    for table_strat in TABLE_STRATEGIES:
        for size in CHUNK_SIZES:
            exp_db_path = os.path.join(CHROMA_BASE_PATH, f"chunk{size}_{table_strat}")
            
            try:
                result, chunk_stat = run_single_experiment(
                    pdf_path=SAMPLE_FILE,
                    chunk_size=size,
                    table_strategy=table_strat,
                    golden_df=golden_df,
                    exp_chroma_path=exp_db_path
                )
                results.append(result)
                chunk_stats.append(chunk_stat)
                
            except Exception as e:
                print(f"  Error: {e}")
                import traceback
                traceback.print_exc()
    
    print("\n" + "=" * 60)
    print("모든 실험 완료!")
    print("=" * 60)
    
elif not TESTSET_READY:
    print("\n Golden Testset을 먼저 작성해주세요!")
    print(f"   파일 위치: {GOLDEN_TESTSET_PATH}")
else:
    print("\n 샘플 PDF 파일이 없습니다.")

## 7. 결과 분석 (Analysis)
Chunk Size와 Table Strategy에 따른 Recall 변화를 시각화합니다.


In [ ]:
# 결과 분석
df_results = pd.DataFrame(results)
df_chunks = pd.DataFrame(chunk_stats)

if not df_results.empty:
    print("=" * 60)
    print("[Result] 실험 결과 요약")
    print("=" * 60)
    
    # 1. Context Recall 비교 (핵심 메트릭)
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1-1. Recall by Config (Bar Chart)
    pivot_recall = df_results.pivot(index="chunk_size", columns="table_strategy", values="context_recall")
    pivot_recall.plot(kind="bar", ax=axes[0], title="Context Recall by Config", colormap="viridis")
    axes[0].set_ylabel("Context Recall")
    axes[0].set_xlabel("Chunk Size")
    axes[0].legend(title="Table Strategy")
    axes[0].axhline(y=0.85, color='r', linestyle='--', label='Target (0.85)')
    
    # 1-2. Chunk Count by Config
    pivot_chunks = df_results.pivot(index="chunk_size", columns="table_strategy", values="num_chunks")
    pivot_chunks.plot(kind="bar", ax=axes[1], title="Number of Chunks", colormap="plasma")
    axes[1].set_ylabel("Chunk Count")
    axes[1].set_xlabel("Chunk Size")
    
    # 1-3. Latency by Config
    pivot_latency = df_results.pivot(index="chunk_size", columns="table_strategy", values="latency_total")
    pivot_latency.plot(kind="bar", ax=axes[2], title="Total Latency (s)", colormap="coolwarm")
    axes[2].set_ylabel("Latency (seconds)")
    axes[2].set_xlabel("Chunk Size")
    
    plt.tight_layout()
    plt.savefig("../data/experiments/exp01_results.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    # 2. 상세 결과 테이블
    print("\n[Details] 상세 결과:")
    print(df_results.to_string(index=False))
    
    # 3. 청크 통계
    print("\n[Stats] 청크 통계:")
    print(df_chunks[['config', 'num_chunks', 'avg_length', 'std_length']].to_string(index=False))
    
    # 4. 최적 설정 선택
    best_idx = df_results['context_recall'].idxmax()
    best_config = df_results.loc[best_idx]
    
    print("\n" + "=" * 60)
    print("[Best] 최적 설정 (Best Config)")
    print("=" * 60)
    print(f"  Config: {best_config['config']}")
    print(f"  Context Recall: {best_config['context_recall']:.4f}")
    print(f"  Chunk Count: {best_config['num_chunks']}")
    print(f"  Total Latency: {best_config['latency_total']:.2f}s")
    
else:
    print("[Warning] 결과가 없습니다. 실험을 먼저 실행하세요.")

## 8. 결론 및 선정 (Conclusion & Selection)

### 8.1 실험 결과 분석
- **최고 성능**: `chunk=500_table=layout` (Recall **0.7333**)
- **Chunk Size**: 작은 청크(500)가 더 정밀한 검색 결과를 보여줌. (500 > 2000 > 1000)
  - 500 Layout (0.73) vs 2000 Text (0.71)로 경합했으나, 500 Layout이 소폭 우세.
- **Table Strategy**: `Layout` 전략이 500 청크에서 뚜렷한 이득을 줌 (+0.05 Recall vs Text).

### 8.2 인사이트 (Findings)
1. **Small & Structured**: RFP의 세부 요건을 찾을 때는 문맥을 너무 길게 잡는(2000) 것보다, **표 구조를 유지한 채 작게(500) 자르는 것**이 유리함.
2. **Layout의 중요성**: 500 토큰 구간에서 Text(0.68) vs Layout(0.73) 차이는 큼. 표 태그/마크다운이 섞이더라도 구조적 의미가 보존될 때 LLM 판단력이 좋아짐.
3. **Latency**: 청크 수가 많아(611개) Latency가 85초대로 다소 증가했으나, 정확도를 위해 감수할 만한 수준.

### 8.3 최종 선정 (Final Decision)
```yaml
Selected Parameters:
- Chunk Size: 500
- Table Strategy: layout
- 근거: Grid Search 최고 Recall (0.7333)
```

### 8.4 적용 계획
- `configs/prod.yaml` 설정을 즉시 업데이트하여 운영 파이프라인에 반영.


## 9. 리포트 저장 (Save)


In [ ]:
import json
from datetime import datetime

# 결과 저장
if results:
    report_path = "../data/experiments/exp01_report.json"
    
    # Best config 추출
    best_idx = df_results['context_recall'].idxmax()
    best = df_results.loc[best_idx].to_dict()
    
    report = {
        "meta": {
            "experiment": "Exp-01 Chunking Optimization",
            "version": "v2 (Human-labeled)",
            "date": datetime.now().isoformat(),
            "sample_file": SAMPLE_FILE,
            "num_test_cases": len(golden_df) if TESTSET_READY else 0,
        },
        "best_config": {
            "chunk_size": int(best['chunk_size']),
            "table_strategy": best['table_strategy'],
            "context_recall": float(best['context_recall']),
        },
        "results": results,
        "chunk_stats": chunk_stats,
    }
    
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"[Saved] Report saved: {report_path}")
    
    # CSV도 저장 (분석 편의)
    df_results.to_csv("../data/experiments/exp01_results.csv", index=False, encoding="utf-8-sig")
    print(f"[Saved] CSV saved: ../data/experiments/exp01_results.csv")
else:
    print("[Warning] 저장할 결과가 없습니다.")